## Jupyter Notebook for RINGO Task 1.3     


### This Jupyter Notebook provides analyses of model results for RINGO Task 1.3 and ICOS measurement time series for 2017 and 2018.

### The analysis consists of 3 parts and is started by selecting station, time range.


<a id='Top'></a>
### 1. Test sampling strategy on 1-hourly STILT time series
- Highest level at ICOS tower sites
- 1-hourly STILT results currently available for 2017 and 2018 at:
    - GAT 341m 
    - OPE 120m 
    - KRE 250m 
    - HPB 131m 
    - HTM 150m 
- selection criteria for QC
    - minimum variability of total CO2 during afternoon hours (12-15 local time)
    - use range between min and max in this time period
    - set threshold 'var_limit'
- selection of 'target' events
    - select high fossil fuel CO2 events (only available in model results)
        - set theshold 'high'
    - select background conditions with low fossil fuel CO2 (only available in model results)
        - set threshold 'low'
    - select CO offset (difference signal - background) as indicator for high fossil fuel CO2 event
        - background defined as lowest value in last 7 days
        - set theshold for offset 'highco'
- restrict selection to 13:00 local time (12 UTC) 
- aggregated footprints and contributions (emission x footprint)
    - monthly aggregation of 13:00 LT
    - low ffCO2
    - high ffCO2
    - high CO
    
### 2. Comparison ICOS observation time series with STILT results
- currently only ICOS level 2 data is included, downloaded from ICOS Carbon Portal to local folder
    - separate files for CO2, CO (and meteo)
    - GAT 341 m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
    - OPE 120 m for 2017 (Jan-Nov) and 2018 (Jan-Dec)
    - KRE 250m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
    - HPB 131m for 2017 (Feb-Dec) and 2018 (Jan-Dec)
    - HTM 150m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
- STILT results from standard footprint tool (3-hourly version) complemented with specific runs for RINGO Task1.3 (1-hourly)
- comparison of time series as monthly plots


### 3. Apply selection to ICOS observation time series
- Highest level at ICOS tower sites
    - GAT 341 m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
    - OPE 120 m for 2017 (Jan-Nov) and 2018 (Jan-Dec)
    - KRE 250m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
    - HPB 131m for 2017 (Feb-Dec) and 2018 (Jan-Dec)
    - HTM 150m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
- selection criteria for QC (same as for model results)
    - minimum variability of total CO2 during afternoon hours (12-15 local time)
    - use range between min and max in this time period
    - set threshold 'var_limit'
- selection criteria for QC based on hourly standard deviation provided in ICOS data set
    - set threshold 'stdev_limit'
- selection of 'target' events
    - select CO offset (difference signal - background) as indicator for high fossil fuel CO2 event
        - background defined as lowest value in last 7 days
        - set theshold for offset 'highco'
- restrict selection to 13:00 local time (12 UTC) 


###  [Goto interactive selection of station, time range and thresholds](#Selection)



### Preparations
#### Import tools 

In [1]:
# import required libaries
import netCDF4 as cdf
import numpy as np
import datetime as dt
import os
import pandas as pd
import matplotlib.pyplot as p
from cartopy import config
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE
import cartopy.feature as cfeature
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from dateutil.rrule import rrule, MONTHLY
from IPython.core.display import display, HTML 
from ipywidgets import interact, interact_manual, Dropdown, SelectMultiple, HBox, VBox, Button, Output, FloatText
from IPython.display import clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

# define colors
orange='#ff8c00'
lime='#00ff00'
aqua='#00ffff'
brown='#663300'
lightgray="#C0C0C0"
gray="#808080"
cb_lightgreen='#b2df8a'
cb_green='#33a02c'
cb_lightblue='#a6cee3'
cb_blue='#1f78b4'
royal='#4169E1'

#### Basic setting and definition of subroutines

In [2]:
# settings
# path to RINGO specific results
path='/opt/stiltdata/RINGO/'
# path to RINGO specific STILT results (1-hourly footprints and concentrations for selected stations)
path_stilt=path+'T1.3/STILT/'
# create directory for storing plots - if not already exists
path_plots = 'Figures/'
if not os.path.exists(path_plots):
    os.makedirs(path_plots)

#### Load basic routines for handling ICOS time series and STILT model results (3-hourly)

In [3]:
%run modules/STILT_modules_plus.ipynb
#%run modules/STILT_modules_v2.5.ipynb
#%run modules/ICOS_atmObs_modules_v2.5.ipynb

Functions defined for handling STILT output:
atc_query
atc_stationlist
available_STILT_dictionary
byte2string
checklib
clear_output
createDatetimeObjList
create_STILT_dictionary
display
get_ICOS_filename
get_ICOS_list
get_station_class
icosDatadf
icosMetadatadf
is_number
lonlat_2_ixjy
plot_available_STILT
read_ICOS_zipfile
read_STILT_dictionary
read_emissions
read_stilt_timeseries
str2dataframe
unzip


In [4]:
# Read dictionary with all stations available in STILT - for 3-hourly STILT results only
# Dictionary contains information on
# - STILT station id
# - Station coordinates (latitude, longitude)
# - Altitude of tracer release in STILT simultation
# - STILT location identifier
# - Station name - if available

stilt_stations = create_STILT_dictionary()

# to list all station with coordinates etc. uncomment next line
#read_STILT_dictionary()

#### Add specific routines for this notebook here

In [5]:
# function to read STILT time series
def read_stilt_ts_T13(station,year,loc_ident):
    filename=path_stilt+'Results_saved/'+station+'/stiltresult'+str(year)+'x'+loc_ident+'.csv'
    df= pd.read_csv(filename,delim_whitespace=True)
    df.date = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    df.name = station
    df.model = 'STILT'
    df['wind.speed']=np.sqrt((df['wind.u']**2)+(df['wind.v']**2))
    df['co2.ff']=df['co2.fuel.coal']+df['co2.fuel.oil']+df['co2.fuel.gas']
    df.set_index(['date'],inplace=True)
    #print(df)
    return df

In [6]:
# function to read meteo observation files
def read_ICOS_atm_meteo_downloaded(filename):
    Names=['Site','SamplingHeight','Year','Month','Day','Hour','Minute','DecimalDate',
           'AP','Stdev','NbPoints','Flag','QualityId',
           'RH','Stdev','NbPoints','Flag','QualityId',
           'AT','Stdev','NbPoints','Flag','QualityId',
           'WD','Stdev','NbPoints','Flag','QualityId',
           'WS','Stdev','NbPoints','Flag','QualityId','InstrumentId']
    df = pd.read_csv(filename,delimiter=';',comment='#',header=None,names=Names)
    df.index = pd.to_datetime(df[['Year','Month','Day','Hour','Minute']])
    #print(df)
    return df

In [7]:
# function to read EDGAR emissions
def read_emissions(filename):
    
    # read EDAGR anthropogenic emissions
    # latitude and longitude are for lower left corner of grid cell
    f = cdf.Dataset(filename)
    #print(f)
    emis=f.variables["emission"][:,:,:] 
    lon_ll=f.variables["lon"][:]
    lat_ll=f.variables["lat"][:]
    time=f.variables["time"][:]

    #shift lat and lon to cell center
    dlon=np.abs(lon_ll[2]-lon_ll[1])
    dlat=np.abs(lat_ll[2]-lat_ll[1])
    emis_lon=lon_ll+0.5*dlon
    emis_lat=lat_ll+0.5*dlat

    unit="["+f.variables["emission"].units+"]"
    
    datetime=cdf.num2date(f.variables["time"][:],units=f.variables["time"].units)
    
    f.close()
    
    return emis, emis_lon, emis_lat, datetime, unit


In [8]:
# plot footprints for selected station and time range
def plot_footprints(station, station_lat, station_lon, loc_ident, daterange, 
                    fxe_vmax=2.0, ident='', secsplit=False, pngfile=''):

    #path to RINGO specific 1-hourly STILT footprints
    path_fp=path_stilt+'Footprints/'
    #path to STILT footprints from the Footprint Tool (3-hourly only)
    path_fp_2='/opt/stiltdata/fsicos2/stilt/Footprints/'

    # loop over all dates and read netcdf files
    # latitude, longitude are for grid center
    fp=[]
    nfp=0
    first = True
    for dd in daterange:
        date_ident=str(dd.year)+'x'+str(dd.month).rjust(2, '0')+'x'+str(dd.day).rjust(2, '0')+'x'+str(dd.hour).rjust(2, '0')
        filename=station+'/foot'+date_ident+'x'+loc_ident+'_aggreg.nc'
        #print(filename)
        read = False
        if os.path.isfile(path_fp+filename):
            f_fp = cdf.Dataset(path_fp+filename)
            read = True
        elif os.path.isfile(path_fp_2+filename):
            f_fp = cdf.Dataset(path_fp_2+filename)
            read = True
        if (read):
            if (first):
                fp=f_fp.variables['foot'][:,:,:]
                lon=f_fp.variables['lon'][:]
                lat=f_fp.variables['lat'][:]
                first = False
            else:
                fp=fp+f_fp.variables['foot'][:,:,:]
            f_fp.close()
            nfp+=1
        else:
            print('files do not exist: ',path_fp+filename,' ',path_fp_2+filename)
            
    if nfp > 0:
        mfp=fp/nfp
        print(str(nfp)+' footprints found'+'   '+ident)
    else:
        print('no footprints found'+'   '+ident)
        return
        
        

    # plot aggregated footprint

    # select colormap
    #cmap = p.get_cmap('YlOrRd')
    #cmap = p.get_cmap('inferno_r')
    #cmap = p.get_cmap('bone_r')
    cmap = p.get_cmap('GnBu')
    #cmap = p.get_cmap('gist_heat_r')
    
    # select marker color
    #mcolor = 'b'
    mcolor = 'r'
    
    # select value range for logarithmic footprints  
    vmin=-7
    vmax=-1
    
    #grid cell index of station 
    ix,jy = lonlat_2_ixjy(station_lon,station_lat,lon,lat)
    #print(station,station_lon,station_lat,ix,jy)

    # define zoom area around station grid cell
    i1 = np.max([ix-65,0])
    i2 = np.min([ix+45,400])
    j1 = np.max([jy-70,0])
    j2 = np.min([jy+70,480])


    lon_z=lon[i1:i2]
    lat_z=lat[j1:j2]
    fp_z=fp[0,j1:j2,i1:i2]

    fig = p.figure(figsize=(17,17))
    
    # Set scale for features from Natural Earth
    #NEscale = '110m'
    NEscale = '50m'
    #NEscale = '10m'
    
    # Create a feature for Countries at 1:50m from Natural Earth
    countries = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale=NEscale,
        facecolor='none')

    # set up a map
    ax = p.subplot(2, 2, 1, projection=ccrs.PlateCarree())
    img_extent = (lon_z.min(), lon_z.max(), lat_z.min(), lat_z.max())
    ax.set_extent([lon_z.min(), lon_z.max(), lat_z.min(), lat_z.max()],crs=ccrs.PlateCarree())
    ax.add_feature(countries, edgecolor='black', linewidth=0.3)

    im = ax.imshow(np.log10(fp_z),interpolation='none',origin='lower', extent=img_extent,cmap=cmap,vmin=vmin,vmax=vmax)
    cbar=p.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend='both')
    cbar.set_label('surface influence log$_{10}$ [ppm / ($\mu$mol / m$^{2}$s)]')
    p.title('{} aggregated footprints '.format(nfp)+'\n'+'station: '+station+'     '+np.min(daterange).strftime('%Y-%m-%d')
            +' -- '+np.max(daterange).strftime('%Y-%m-%d')+'\n'+ident)#+'\n'+'station: '+station+'  '+loc_ident)
    ax.text(0.01, -0.27, 'min: %.5e' % np.nanmin(fp_z), horizontalalignment='left',transform=ax.transAxes)
    ax.text(0.99, -0.27, 'max: %.5e' % np.nanmax(fp_z), horizontalalignment='right',transform=ax.transAxes)
 
    # station location
    ax.plot(station_lon,station_lat,'+',color=mcolor,ms=10,markeredgewidth=2,transform=ccrs.PlateCarree())

    # total annual emissions original EDGAR 4.3 for base year 2010
    path_edgar=path+'/Emissions/'
    filename='EDGARv4.3_2010_total.nc'
    emis, lon, lat, dd, unit = read_emissions(path_edgar+filename)

    # footprint * emission 
    fp_emis=fp*emis
    fp_emis[fp<=0.0]=np.nan
    
    fp_emis_z=fp_emis[0,j1:j2,i1:i2]

    ax = p.subplot(2, 2, 2, projection=ccrs.PlateCarree())
    img_extent = (lon_z.min(), lon_z.max(), lat_z.min(), lat_z.max())
    ax.set_extent([lon_z.min(), lon_z.max(), lat_z.min(), lat_z.max()],crs=ccrs.PlateCarree())
    ax.add_feature(countries, edgecolor='black', linewidth=0.3)

    im = ax.imshow((fp_emis_z)[:,:],interpolation='none',origin='lower',extent=img_extent,vmin=0,vmax=fxe_vmax,cmap=cmap)
    cbar=p.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend='max')
    cbar.set_label('surface influence [ppm]')
    p.title('Footprints x Emissions '+'\n'+'station: '+station+'     '+np.min(daterange).strftime('%Y-%m-%d')
            +' -- '+np.max(daterange).strftime('%Y-%m-%d')+'\n'+ident)#+'\n'+'station: '+station+'  '+loc_ident)
    ax.text(0.01, -0.27, 'min: %.5e' % np.nanmin(fp_emis_z), horizontalalignment='left',transform=ax.transAxes)
    ax.text(0.99, -0.27, 'max: %.5e' % np.nanmax(fp_emis_z), horizontalalignment='right',transform=ax.transAxes)

    #station location
    ax.plot(station_lon,station_lat,'+',color=mcolor,ms=10,markeredgewidth=2,transform=ccrs.PlateCarree())
    

    if secsplit:
        # emissions from enery production only, original EDGAR 4.3 for base year 2010
        path_edgar=path+'/Emissions/'
        filename='EDGARv4.3_2010_ENE.nc'
    
        emis2, lon, lat, dd, unit = read_emissions(path_edgar+filename)

        # footprint * emission 
        fp_emis2=fp*emis2
        fp_emis2[fp<=0.0]=np.nan
    
        fp_emis2_z=fp_emis2[0,j1:j2,i1:i2]

        ax = p.subplot(2, 2, 3, projection=ccrs.PlateCarree())
        img_extent = (lon_z.min(), lon_z.max(), lat_z.min(), lat_z.max())
        ax.set_extent([lon_z.min(), lon_z.max(), lat_z.min(), lat_z.max()],crs=ccrs.PlateCarree())
        ax.add_feature(countries, edgecolor='black', linewidth=0.3)

        im = ax.imshow((fp_emis2_z)[:,:],interpolation='none',origin='lower',extent=img_extent,vmin=0,vmax=fxe_vmax,cmap=cmap)
        cbar=p.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend='max')
        cbar.set_label('surface influence [ppm]')
        p.title('Footprints x Energy-Emissions '+'\n'+'station: '+station+'     '
                +np.min(daterange).strftime('%Y-%m-%d')+' -- '+np.max(daterange).strftime('%Y-%m-%d')
                +'\n'+ident)#+'\n'+'station: '+station+'  '+loc_ident)
        ax.text(0.01, -0.27, 'min: %.5f' % np.nanmin(fp_emis2_z), horizontalalignment='left',transform=ax.transAxes)
        ax.text(0.99, -0.27, 'max: %.5f' % np.nanmax(fp_emis2_z), horizontalalignment='right',transform=ax.transAxes)

        #station location
        ax.plot(station_lon,station_lat,'+',color=mcolor,ms=10,markeredgewidth=2,transform=ccrs.PlateCarree())
    
        # big cities in Europe
        ax.scatter(df_bigCities.longitude,df_bigCities.latitude,marker='o',color='k',facecolors='none',s=100,transform=ccrs.PlateCarree())

      
        # emissions from all emissions except enery production, original EDGAR 4.3 for base year 2010
        path_edgar=path+'/Emissions/'
        filename='EDGARv4.3_2010_nonENE.nc'
    
        emis3, lon, lat, dd, unit = read_emissions(path_edgar+filename)

        # footprint * emission 
        fp_emis3=fp*emis3
        fp_emis3[fp<=0.0]=np.nan
    
        fp_emis3_z=fp_emis3[0,j1:j2,i1:i2]

        ax = p.subplot(2, 2, 4, projection=ccrs.PlateCarree())
        img_extent = (lon_z.min(), lon_z.max(), lat_z.min(), lat_z.max())
        ax.set_extent([lon_z.min(), lon_z.max(), lat_z.min(), lat_z.max()],crs=ccrs.PlateCarree())
        ax.add_feature(countries, edgecolor='black', linewidth=0.3)

        im = ax.imshow((fp_emis3_z)[:,:],interpolation='none',origin='lower',extent=img_extent,vmin=0.000001,vmax=fxe_vmax,cmap=cmap)
        cbar=p.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend='max')
        cbar.set_label('surface influence [ppm]')
        p.title('Footprints x nonEnergy-Emissions '+'\n'+'station: '+station+'     '
                +np.min(daterange).strftime('%Y-%m-%d')+' -- '+np.max(daterange).strftime('%Y-%m-%d')
                +'\n'+ident)#+'\n'+'station: '+station+'  '+loc_ident)
        ax.text(0.01, -0.27, 'min: %.5f' % np.nanmin(fp_emis3_z), horizontalalignment='left',transform=ax.transAxes)
        ax.text(0.99, -0.27, 'max: %.5f' % np.nanmax(fp_emis3_z), horizontalalignment='right',transform=ax.transAxes)

        #station location
        ax.plot(station_lon,station_lat,'+',color=mcolor,ms=10,markeredgewidth=2,transform=ccrs.PlateCarree())
    
        # big cities in Europe
        ax.scatter(df_bigCities.longitude,df_bigCities.latitude,marker='o',color='k',facecolors='none',s=100,transform=ccrs.PlateCarree())#,latlon=True)

        
    p.tight_layout()
    p.show()
    if len(pngfile)>0:
        fig.savefig(pngfile+'.png',dpi=100)
    p.close()

In [9]:
# function to apply temporal sampling pf time series + plot results
def plot_stilt_ts_selection(df,start_date,end_date,
                            obs=None,meteo=None,pngfile='',high=5.0, low=1.5, highco=0.05, var_limit=1.,summary=False):
    
    # selections

    # afternoon sampling
    # local time 12-15=> utc 11-14
    dd1=11
    dd2=14
    df_day = df.loc[(df['hour'] >= dd1) & (df['hour'] <= dd2)]
    # range in afternoon values (max-min)
    df_range = df_day.resample('D',loffset='12H').max() - df_day.resample('D',loffset='12H').min()
    df_range.loc[(df_range['co2.stilt'] <= 0)]=np.nan
    
    # low variability selection for QC  (use range as variability)
    df_var = df_range.loc[(df_range['co2.stilt'] < var_limit)]
    df_var3 = df_var.iloc[::3, :]

    # single noon value at 12 UTC => 13 LT for CET
    d1=12
    d2=12
    df_noon = df.loc[(df['hour'] >= d1) & (df['hour'] <= d2)]

    # low fossil fuel CO2 => background
    df_low=df_noon.loc[(df_noon['co2.fuel.coal']+df_noon['co2.fuel.oil']+df_noon['co2.fuel.gas'] < low)]
    
    # high ffCO2 => target
    df_high=df_noon.loc[(df_noon['co2.fuel.coal']+df_noon['co2.fuel.oil']+df_noon['co2.fuel.gas'] > high)]

    # select background based on wind direction
    # example for Gartow
    df_nordsee=df_low.loc[(df_low['wind.dir'] < 360) & (df_low['wind.dir'] > 270)]

    df_noon.name = df.name
    df_noon.model = df.model

    # select only every 3rd noon value
    df_noon3 = df_noon.iloc[::3, :]

    # select lowest value for previous 7 days
    df_min = df.rolling('7d',closed='right').min()
    # only noon value (select based on date info in original time series)
    df_min_noon = df_min.loc[(df['hour'] >= d1) & (df['hour'] <= d2)]

    # difference between 7-day minimum and value at noon
    df_offset=df_noon.subtract(df_min_noon)

    # high CO offset as indicator for high ffCO2
    df_highco=df_noon.loc[(df_offset['co.stilt'] > highco)]
    
    if summary:
        # print summary 
        n_highffco2 = df_high.groupby(pd.TimeGrouper(freq='M')).count()
        display(HTML('<br> <p style="font-size:15px;font-weight:bold;color:royalblue;">Events with ffCO2 offset > '+str(high)+' ppm</p>'))
        display(HTML(n_highffco2['co2.stilt'].to_frame(name='counts per month').to_html())) 
        
        n_highco = df_highco.groupby(pd.TimeGrouper(freq='M')).count()
        display(HTML('<br> <p style="font-size:15px;font-weight:bold;color:royalblue;">Events with CO offset > '+str(highco)+' ppm</p>'))
        display(HTML(n_highco['co2.stilt'].to_frame(name='counts per month').to_html())) 

        n_var = df_var.groupby(pd.TimeGrouper(freq='M')).count()
        display(HTML('<br> <p style="font-size:15px;font-weight:bold;color:royalblue;">Events with small daytime CO2 difference < '+str(var_limit)+' ppm</p>'))
        display(HTML(n_var['co2.stilt'].to_frame(name='counts per month').to_html())) 
        display(HTML('<br> <br>'))


    #plot time series
    
    fig = p.figure(figsize=(15,14))
    ax = fig.add_subplot(5,1,1)
    p.plot(df.index,df['co2.stilt'],'.',color=lightgray,label=df.name+' co2.stilt')
    p.plot(df_noon.index,df_noon['co2.stilt'],'.',color='b',label=df.name+' co2.stilt '+str(d1+1)+' LT')
    p.plot(df_low.index,df_low['co2.stilt'],'o',color='y',label='low co2.fuel < '+str(low)+' '+str(d1+1)+' LT')
    p.plot(df_high.index,df_high['co2.stilt'],'o',color='r',label='high co2.fuel >'+str(high)+' '+str(d1+1)+' LT')
    p.plot(df_highco.index,df_highco['co2.stilt'],'+',color='m',label='high co offset >'+str(highco)+' '+str(d1+1)+' LT',
           markersize=10,markeredgewidth=2)
    p.title(df.name+'  '+str(df['latstart'][0])+'$^\circ$N'+'  '+str(df['lonstart'][0])+'$^\circ$E')
    ax.set_xlim(start_date,end_date)
    #ax.set_ylim(-10,30)
    ax.set_ylabel('CO$_2$  [ppm]')
    ax.grid(axis='x')
    ax.grid(axis='y')
    ax.legend(loc='best',ncol=8)#loc='upper right')

    ax = fig.add_subplot(5,1,2)
    p.plot([start_date,end_date],[highco,highco],':',color='k',linewidth=0.5)
    p.plot(df.index,df['co.stilt'],'.',color=lightgray,label='co.stilt')
    p.plot(df_noon.index,df_noon['co.stilt'],'.',color='b',label='co.stilt '+str(d1+1)+' LT')
    #p.plot(df_noon3.index,df_noon3['co.stilt'],'.',color='c',label='co.stilt every 3rd day '+str(d1+1)+' LT')
    p.plot(df_min_noon.index,df_min_noon['co.stilt'],'-',color='r',label='co.stilt 7 day minimum '+str(d1+1)+' LT')
    p.plot(df_offset.index,df_offset['co.stilt'],'o',color='m',label='co offset '+str(d1+1)+' LT')
    #p.title(df.name+'  '+str(df['latstart'][0])+'$^\circ$N'+'  '+str(df['lonstart'][0])+'$^\circ$E')
    ax.set_xlim(start_date,end_date)
    ##ax.set_ylim(390,410)
    ax.set_ylabel('regional CO [ppm]')
    ax.grid(axis='x')
    ax.grid(axis='y')
    ax.legend(loc='best',ncol=8)#loc='upper right')

    ax = fig.add_subplot(5,1,3)
    p.plot([start_date,end_date],[high,high],':',color='k',linewidth=0.5)
    p.plot(df_noon.index,df_noon['co2.bio'],'.',color='g',label=df.name+' co2.bio '+str(d1+1)+' LT')
    p.plot(df_noon.index,df_noon['co2.ff'],'.',color='r',label=df.name+' co2.ff '+str(d1+1)+' LT')
    p.plot(df_low.index,df_low['co2.ff'],'o',color='y',label='co2.ff low < '+str(low))
    p.plot(df_high.index,df_high['co2.ff'],'o',color='r',label='co2.ff high >'+str(high))
    p.plot(df_highco.index,df_highco['co2.ff'],'+',color='m',label='co2.ff high co offset >'+str(highco),
           markersize=10,markeredgewidth=2)
    #p.title(df.name+'  '+str(df['latstart'][0])+'$^\circ$N'+'  '+str(df['lonstart'][0])+'$^\circ$E')
    ax.set_xlim(start_date,end_date)
    ax.set_ylim(-10,30)
    ax.set_ylabel('CO$_2$ components  [ppm]')
    ax.grid(axis='x')
    ax.grid(axis='y')
    ax.legend(loc='best',ncol=8)#loc='upper right')

    ax = fig.add_subplot(5,1,4)
    p.plot(df_range.index,df_range['co2.stilt'],'.',color='r',label='range co2.stilt '+str(dd1+1)+'-'+str(dd2+1)+' LT')
    p.plot(df_var.index,df_var['co2.stilt'],'o',color='b',label='range co2.stilt < '+str(var_limit)+' '+str(dd1+1)+'-'+str(dd2+1)+' LT')
    ax.set_xlim(start_date,end_date)
    #ax.set_ylim(390,410)
    ax.set_ylabel('CO$_2$  [ppm]')
    ax.grid(axis='x')
    ax.grid(axis='y')
    ax.legend(loc='best',ncol=8)#loc='upper right')

    ax = fig.add_subplot(5,1,5)
    p.plot([start_date,end_date],[0,0],color=gray,linewidth=0.5)
    p.plot([start_date,end_date],[90,90],color=gray,linewidth=0.5)
    p.plot([start_date,end_date],[180,180],color=gray,linewidth=0.5)
    p.plot([start_date,end_date],[270,270],color=gray,linewidth=0.5)
    p.plot([start_date,end_date],[360,360],color=gray,linewidth=0.5)
    p.plot(df.index,df['wind.dir'],'.',color=lightgray,label='stilt wind dir')
    ax.set_xlim(start_date,end_date)
    ax.set_ylim(0,360)
    ax.set_ylabel('wind direction')
    ax.grid(axis='x')
    #ax.grid(axis='y')
    ax.legend(loc='best',ncol=8)#loc='upper right')

    
    p.tight_layout()
    p.show()
    if len(pngfile)>0:
        fig.savefig(pngfile+'.png',dpi=100)
    p.close()

    df_select_high=df_high[(df_high.index >= start_date) & (df_high.index <= end_date)]
    df_select_highco=df_highco[(df_highco.index >= start_date) & (df_highco.index <= end_date)]
    df_select_low=df_low[(df_low.index >= start_date) & (df_low.index <= end_date)]
    df_select_noon=df_noon[(df_noon.index >= start_date) & (df_noon.index <= end_date)]
    df_select_noon3=df_noon3[(df_noon3.index >= start_date) & (df_noon3.index <= end_date)]
    df_select_var=df_var[(df_var.index >= start_date) & (df_var.index <= end_date)]
        
    return df_select_low, df_select_high, df_select_highco, df_select_noon, df_select_noon3, df_select_var


In [10]:
def all_selection_plots(station,sdate,edate,loc_ident,station_lat,station_lon,
                        high,low,highco,var_limit,summary=False,plot_foot=True):
    
    # read 1-hourly STILT results (RINGO specific STILT runs)
    df_stilt_1hr = pd.DataFrame()
    for year in range(sdate.year,edate.year+1):
        df = read_stilt_ts_T13(station,year,loc_ident)
        df_stilt_1hr = df_stilt_1hr.append(df)
        df_stilt_1hr.name = df.name
        df_stilt_1hr.model = df.model
        
    for yrmon in rrule(MONTHLY, dtstart=sdate, until=edate):

        start_date = dt.datetime(yrmon.year,yrmon.month,1,0)
        if yrmon.month >= 12:
            end_date = dt.datetime(yrmon.year+1,1,1,0)
        else:
            end_date = dt.datetime(yrmon.year,yrmon.month+1,1,0)
        end_date = end_date - dt.timedelta(hours=1)
    
        # return time serie selection
        pngfile='selection_STILT_'+station+'_'+str(sdate.year)+str(sdate.month).zfill(2)
        df_select_noon_low, df_select_noon_high, df_select_noon_highco, df_select_noon, df_select_3rdnoon, df_select_noon_var = \
                                 plot_stilt_ts_selection(df_stilt_1hr,start_date,end_date,pngfile=path_plots+pngfile,
                                                         low=low,high=high,highco=highco,summary=summary)
        summary = False

        if plot_foot:
            # aggregated footprints

            # set upper limit for color scale in 'footprint x emissions' map
            # e.g. fxe_vmax=0.16
            # in all other maps the color scale is set automatically
            # all footprints at noontime 
            selected=df_select_noon.index
            pngfile='FP_noon_STILT_'+station+'_'+str(sdate.year)+str(sdate.month).zfill(2)
            plot_footprints(station,station_lat,station_lon,loc_ident,selected,
                            fxe_vmax=0.14,ident='all 13 LT',pngfile=path_plots+pngfile)
        
            # footprints at noontime, every 3rd day only 
            selected=df_select_3rdnoon.index
            pngfile='FP_noon3_STILT_'+station+'_'+str(sdate.year)+str(sdate.month).zfill(2)
            plot_footprints(station,station_lat,station_lon,loc_ident,selected,
                            fxe_vmax=0.14,ident='13 LT every 3rd day',pngfile=path_plots+pngfile)
        
            # footprints for situations with low variability around noon
            selected=df_select_noon_var.index
            pngfile='FP_var_STILT_'+station+'_'+str(sdate.year)+str(sdate.month).zfill(2)
            plot_footprints(station,station_lat,station_lon,loc_ident,selected,
                            fxe_vmax=0.14,ident='low variability',pngfile=path_plots+pngfile)

            # footprints for low ffCO2
            selected=df_select_noon_low.index
            pngfile='FP_lowffCO2_STILT_'+station+'_'+str(sdate.year)+str(sdate.month).zfill(2)
            plot_footprints(station,station_lat,station_lon,loc_ident,selected,
                            fxe_vmax=0.14,ident='low ffCO2',pngfile=path_plots+pngfile)

            # footprints for high ffCO2
            selected=df_select_noon_high.index
            pngfile='FP_highffCO2_STILT_'+station+'_'+str(sdate.year)+str(sdate.month).zfill(2)
            plot_footprints(station,station_lat,station_lon,loc_ident,selected,
                            fxe_vmax=0.14,ident='high ffCO2',secsplit=True,pngfile=path_plots+pngfile)

            # footprints for high CO offset
            selected=df_select_noon_highco.index
            pngfile='FP_highffCO_STILT_'+station+'_'+str(sdate.year)+str(sdate.month).zfill(2)
            plot_footprints(station,station_lat,station_lon,loc_ident,selected,
                            fxe_vmax=0.14,ident='high CO offset',pngfile=path_plots+pngfile)


#### Function for plotting STILT time series together with observations (updated version of basic example in STILT_modules.ipnb)

In [11]:
# function to plot STILT time series (new format of STILT results)
def plot_icos_stilt_timeseries(station,df,start_date,end_date,
                               obs=None,meteo=None,title2='',linestyle = '.',pngfile='',add_tracer=[]):
    #plot time series
    tracer = add_tracer + ['co2']
    tracer = [x.lower() for x in tracer]
    fig = p.figure(figsize=(15,14))

    ax = fig.add_subplot(5,1,1)
    p.plot(df.index,df['co2.stilt'],linestyle,color='b',label='STILT co2.stilt')
    p.plot(df.index,df['co2.background'],linestyle,color='c',label='STILT co2.background')
    if obs is not None and not obs.empty:
        if 'DateTime' in obs:
            p.plot(obs.DateTime,obs['CO2'],linestyle,color='k',label='observation')
        else:
            if 'co2' in obs:
                p.plot(obs.index,obs['co2'],linestyle,color='k',label='observation')
            if 'CO2' in obs:
                p.plot(obs.index,obs['CO2'],linestyle,color='k',label='observation')
    p.title(df.name+'  '+str(df['latstart'][0])+'$^\circ$N'+'  '+str(df['lonstart'][0])+'$^\circ$E'+'    '+title2)
    ax.set_xlim(start_date,end_date)
    ax.set_ylabel('CO$_2$  [ppm]')
    ax.grid(axis='x')
    ax.grid(axis='y')
    ax.legend(loc='upper right')

    ax = fig.add_subplot(5,1,2)
    p.plot(df.index,df['co2.fuel.coal']+df['co2.fuel.oil']+df['co2.fuel.gas'],linestyle,color='r',label='STILT co2.fossil')
    p.plot(df.index,df['co2.bio'],linestyle,color='g',label='STILT co2.bio')
    ax.set_xlim(start_date,end_date)
    #ax.set_ylim(-50,200)
    ax.set_ylabel('CO$_2$ components  [ppm]')
    ax.grid(axis='x')
    ax.grid(axis='y')
    ax.legend(loc='upper right')

    if ('co.stilt' in df) and ('co' in tracer):
        ax = fig.add_subplot(5,1,3)
        p.plot(df.index,df['co.stilt'],linestyle,color='m',label='STILT co.stilt')
        if obs is not None and not obs.empty:
            if 'DateTime' in obs:
                p.plot(obs.index,obs['CO'],linestyle,color='k',label='observation')
            else:
                # select lowest value for previous 7 days
                # only works if DateTime is defined as index
                # in case of many missing values, 'min' is not defined, better use very low quantile 
                df_obs_min = obs.rolling('7d',closed='right',min_periods=1).quantile(0.001) 
                if 'co' in obs:
                    p.plot(obs.index,obs['co']-df_obs_min['co'],linestyle,color='k',label='observation')
                if 'CO' in obs:
                    p.plot(obs.index,obs['CO']-df_obs_min['CO'],linestyle,color='k',label='observation')               
        #p.title(df.name[0]+'  '+str(df['latstart'][0])+'$^\circ$N'+'  '+str(df['lonstart'][0])+'$^\circ$E'+'    '+title2)
        ax.set_xlim(start_date,end_date)
        ax.set_ylabel('CO  [ppb]')
        ax.grid(axis='x')
        ax.grid(axis='y')
        ax.legend(loc='upper right')

    if ('rn' in df) and ('rn' in tracer):
        ax = fig.add_subplot(5,1,4)
        p.plot(df.index,df['rn'],linestyle,color='y',label='STILT rn')
        if obs is not None and 'rn' in obs and not obs.empty:
            p.plot(obs.index,obs['rn'],linestyle,color='k',label='observation')
        #p.title(df.name[0]+'  '+str(df['latstart'][0])+'$^\circ$N'+'  '+str(df['lonstart'][0])+'$^\circ$E'+'    '+title2)
        ax.set_xlim(start_date,end_date)
        ax.set_ylabel('222RN  [???]')
        ax.grid(axis='x')
        ax.grid(axis='y')
        ax.legend(loc='upper right')

    p.tight_layout()
    p.show()
    if len(pngfile)>0:
        fig.savefig(pngfile+'.png',dpi=100)
    p.close()


In [12]:
# function to plot ICOS time series 

def plot_comparison(istat,sdate,edate,ICOS_files):
    
    # check if ICOS data is available for station
    if len(istat) > 3:
        searchName=istat[0:3]+'_'+str(int(istat[3:]))
    else:
        searchName=istat
    
    # CO2 only
    filenames = [i for i in ICOS_files['CO2'].fileName if searchName in i]
    if not filenames:
        print('No ICOS CO2 data available for station ',searchName)
        df_stationdata=pd.DataFrame({'A' : []})
    else:
        for filename in filenames:
            df_stationmetadata, df_stationdata = read_ICOS_zipfile(filename, 'co2')
    
    # CO only
    filenames = [i for i in ICOS_files['CO'].fileName if searchName in i]
    if not filenames:
        print('No ICOS CO data available for station ',searchName)
        df_stationdata_co=pd.DataFrame({'A' : []})
    else:
        for filename in filenames:
            df_stationmetadata_co, df_stationdata_co = read_ICOS_zipfile(filename, 'co')
            df_stationdata_co.rename(columns={'Stdev': 'StdevCO'}, inplace=True)

    # combine co2 and co measurement data sets into a single data frame
    if not df_stationdata.empty and not df_stationdata_co.empty :
        df_co2 = df_stationdata[['Hour','co2','Stdev']]
        df_co = df_stationdata_co[['co','StdevCO']]
        df_co2['Hour'] = pd.to_numeric(df_co2['Hour'])
        df_co2['co2'] = pd.to_numeric(df_co2['co2'])
        df_co2['Stdev'] = pd.to_numeric(df_co2['Stdev'])
        df_co['co'] = pd.to_numeric(df_co['co'])
        df_co['co'] = df_co['co']/1000.
        df_co['StdevCO'] = pd.to_numeric(df_co['StdevCO'])

        df_obs = pd.merge(df_co2,df_co,left_index=True, right_index=True, how='outer')
    else:
        df_obs = pd.DataFrame({'A' : []})
        
           
    # adjust station name to use STILT name for the rest of this cell, e.g. for Gartow 341m
    if istat == 'GAT341':
        ist = 'GAT344'
    else:
        ist = istat

    # check if STILT results are available for the station 
    if not ist in stilt_stations.keys():
        print('no STILT results for station: ',ist)
    else:
        # read 1-hourly STILT time series, available only for specific stations
        df_stilt = pd.DataFrame()
        for year in range(sdate.year,edate.year+1):
            df = read_stilt_ts_T13(ist,year,stilt_stations[ist]['locIdent'])
            df_stilt = df_stilt.append(df)
            df_stilt.name = df.name
            df_stilt.model = df.model

        if df.empty:
            print('no 1-hourly STILT results for station: ',ist)
        else:
                
            # monthly plots
            for yrmon in rrule(MONTHLY, dtstart=sdate, until=edate):

                start_date = dt.datetime(yrmon.year,yrmon.month,1,0)
                if yrmon.month >= 12:
                    end_date = dt.datetime(yrmon.year+1,1,1,0)
                else:
                    end_date = dt.datetime(yrmon.year,yrmon.month+1,1,0)
                end_date = end_date - dt.timedelta(hours=1)

                timeselect='all'

                # plot time series
                pngfile='comparison_ICOS_STILT_'+ist+'_'+str(start_date.year)+str(start_date.month).zfill(2)
                plot_icos_stilt_timeseries(ist, df_stilt, start_date, end_date, obs = df_obs, 
                                           linestyle='-',add_tracer=['co'],pngfile=path_plots+pngfile)
                

In [13]:
def plot_icos_ts_selection(ist, ICOS_files, highco, var_limit, stdev_limit, start=None, end=None, summary=False):

    # check if ICOS data is available for station
    if len(ist) > 3:
        searchName=ist[0:3]+'_'+str(int(ist[3:]))
    else:
        searchName=ist

    # CO2 only
    filenames = [i for i in ICOS_files['CO2'].fileName if searchName in i]
    if not filenames:
        print('No ICOS CO2 data available for station ',searchName)
        df_stationdata = pd.DataFrame({'A' : []})
    else:
        for filename in filenames:
            df_stationmetadata, df_stationdata = read_ICOS_zipfile(filename, 'co2')

    # CO only
    filenames = [i for i in ICOS_files['CO'].fileName if searchName in i]
    if not filenames:
        print('No CO ICOS data available for station ',searchName)
        df_stationdata_co = pd.DataFrame({'A' : []})
    else:
        for filename in filenames:
            df_stationmetadata_co, df_stationdata_co = read_ICOS_zipfile(filename, 'co')
    
    # Meteo only
    # no meteo for Gartow 341m available -> use lower level at 216m
    if ist == 'GAT341':
        searchName = 'GAT_216'
    filenames = [i for i in ICOS_files['MTO'].fileName if searchName in i]
    if not filenames:
        print('No ICOS Meteo data available for station ',searchName)
        df_stationdata_meteo = pd.DataFrame({'A' : []})
    else:
        for filename in filenames:
            df_stationdata_meteo = read_ICOS_atm_meteo_downloaded(filename)
 
    ### Need to concatenate time series for one station in case of several files !!!

    if not df_stationdata.empty and not df_stationdata_co.empty: 
        # limit for pollution event
        #highco=0.04 (default)
        # low variability based on afternoon CO2 range
        #var_limit=1.0 (default)
        # low variability based on hourly CO2 standard deviation
        #stdev_limit=0.5 (default)
        
        # combine all measurement data in one single dataframe
        df_co2 = df_stationdata[['Month','Hour','co2','Stdev']]
        df_co = df_stationdata_co[['co']]
        df_co2['Hour'] = pd.to_numeric(df_co2['Hour'])
        df_co2['Month'] = pd.to_numeric(df_co2['Month'])
        df_co2['co2'] = pd.to_numeric(df_co2['co2'])
        df_co2['Stdev'] = pd.to_numeric(df_co2['Stdev'])
        df_co['co'] = pd.to_numeric(df_co['co'])
        df_co['co'] = df_co['co']/1000. # convert tp ppb

        df_obs = pd.merge(df_co2,df_co,left_index=True, right_index=True, how='outer')
        
        # afternoon sampling
        # local time 12-15=> utc 11-14
        dd1=11
        dd2=14
        df_obs_day = df_obs.loc[(df_obs['Hour'] >= dd1) & (df_obs['Hour'] <= dd2)]

        # range in afternoon values (max-min)
        df_obs_range = df_obs_day.resample('D',loffset='12H').max() - df_obs_day.resample('D',loffset='12H').min()
        df_obs_range.loc[(df_obs_range['co2'] <= 0)]=np.nan

        # low variability selection for QC  (use range as variability)
        df_obs_var = df_obs_range.loc[(df_obs_range['co2'] < var_limit)]
        df_obs_var3 = df_obs_var.iloc[::3, :]

        df_obs_lowStdev=df_obs.loc[(df_obs['Stdev'] < stdev_limit)]
                        
        # single noon value at 12 UTC => 13 LT for CET
        d1=12
        d2=12
        df_obs_noon = df_obs.loc[(df_obs['Hour'] >= d1) & (df_obs['Hour'] <= d2)]

        # high CO as indicator for high ffCO2
        df_obs_highco=df_obs_noon.loc[(df_obs_noon['co'] > highco)]

        # select only every 3rd noon value
        df_obs_noon3 = df_obs_noon.iloc[::3, :]

        # select lowest value for previous 7 days
        # in case of many missing values, 'min' is not defined, better use very low quantile 
        df_obs_min = df_obs.rolling('7d',closed='right',min_periods=1).quantile(0.001)
        # only noon value (select based on date info in original time series)
        df_obs_min_noon = df_obs_min.loc[(df_obs['Hour'] >= d1) & (df_obs['Hour'] <= d2)]

        # difference between 7-day minimum and value at noon
        df_obs_offset=df_obs_noon.subtract(df_obs_min_noon)

        # high CO offset as indicator for high ffCO2
        df_obs_highco=df_obs_noon.loc[(df_obs_offset['co'] > highco)]
    
        # low standard deviation at noon
        df_obs_lowStdev_noon=df_obs_noon.loc[(df_obs_noon['Stdev'] < stdev_limit)]
    

        # plot time series for each month
        # if start and/or end date are specified in the parameters list, use them
        # otherwise extraxt start and end date of time series
        if start is not None:
            sd = start
        else:
            sd0 = df_obs.index[0]
            sd = dt.datetime(sd0.year,sd0.month,1,0)

        if end is not None:
            ed = end
        else:
            ed0 = df_obs.index[-1]+dt.timedelta(days=32-df_obs.index[-1].day) #round up to next month/year
            ed = dt.datetime(ed0.year,ed0.month,1,0)

        for yrmon in rrule(MONTHLY, dtstart=sd, until=ed):
            start_date = dt.datetime(yrmon.year,yrmon.month,1,0)
            if yrmon.month >= 12:
                end_date = dt.datetime(yrmon.year+1,1,1,0)
            else:
                end_date = dt.datetime(yrmon.year,yrmon.month+1,1,0)
            end_date = end_date - dt.timedelta(hours=1)

            fig = p.figure(figsize=(15,14))
            ax = fig.add_subplot(5,1,1)
            p.plot(df_obs.index,df_obs['co2'],'.',color=lightgray,label=ist+' CO2 obs ')
            p.plot(df_obs_noon.index,df_obs_noon['co2'],'.',color='b',label=ist+' CO2 obs '+str(d1+1)+' LT')
            p.plot(df_obs_highco.index,df_obs_highco['co2'],'+',color='m',label='high co offset >'+str(highco)+' '+str(d1+1)+' LT',markersize=10,markeredgewidth=2)
            p.title(ist)
            ax.set_xlim(start_date,end_date)
            #ax.set_ylim(-10,30)
            ax.set_ylabel('CO$_2$  [ppm]')
            ax.grid(axis='x')
            ax.grid(axis='y')
            ax.legend(loc='best',ncol=8)

            ax = fig.add_subplot(5,1,2)
            p.plot([start_date,end_date],[highco,highco],'--',color='k',linewidth=0.5)
            p.plot(df_obs.index,df_obs['co'],'.',color=lightgray,label='CO obs')
            p.plot(df_obs_noon.index,df_obs_noon['co'],'.',color='b',label='CO obs '+str(d1+1)+' LT')
            p.plot(df_obs_min_noon.index,df_obs_min_noon['co'],'-',color='r',label='CO obs 7 day minimum '+str(d1+1)+' LT')
            p.plot(df_obs_offset.index,df_obs_offset['co'],'o',color='m',label='CO offset '+str(d1+1)+' LT')
            #p.title(ist)
            ax.set_xlim(start_date,end_date)
            ax.set_ylim(-0.05,0.4)
            ax.set_ylabel('CO [ppm]')
            ax.grid(axis='x')
            ax.grid(axis='y')
            ax.legend(loc='best',ncol=8)

            ax = fig.add_subplot(5,1,3)
            p.plot(df_obs.index,df_obs['Stdev'],'.',color=lightgray,label=ist+' CO2 Stdev ')
            p.plot(df_obs_noon.index,df_obs_noon['Stdev'],'.',color=lime,label=ist+' CO2 Stdev '+str(d1+1)+' LT')
            #p.title(ist)
            ax.set_xlim(start_date,end_date)
            ax.set_ylim(-0.5,6)
            ax.set_ylabel('CO$_2$  [ppm]')
            ax.grid(axis='x')
            ax.grid(axis='y')
            ax.legend(loc='best',ncol=8)
            
            ax = fig.add_subplot(5,1,4)
            p.plot([start_date,end_date],[var_limit,var_limit],'--',color='k',linewidth=0.5)
            p.plot([start_date,end_date],[stdev_limit,stdev_limit],'--',color='k',linewidth=0.5)
            p.plot(df_obs_range.index,df_obs_range['co2'],'.',color='r',label='range CO2 '+str(dd1+1)+'-'+str(dd2+1)+' LT')
            p.plot(df_obs_var.index,df_obs_var['co2'],'o',color='b',label='range CO2 < '+str(var_limit)+' '+str(dd1+1)+'-'+str(dd2+1)+' LT')
            p.plot(df_obs_noon.index,df_obs_noon['Stdev'],'.',color=lime,label=ist+' CO2 Stdev '+str(d1+1)+' LT')
            p.plot(df_obs_lowStdev_noon.index,df_obs_lowStdev_noon['Stdev'],'^',color='g',label=ist+' CO2 Stdev < '+str(stdev_limit)+' '+str(d1+1)+' LT')
            ax.set_xlim(start_date,end_date)
            ax.set_ylim(-0.5,6)
            #ax.set_ylim(390,410)
            ax.set_ylabel('CO$_2$ [ppm]')
            ax.grid(axis='x')
            ax.grid(axis='y')
            ax.legend(loc='best',ncol=8)

            #df_stationdata_meteo
            ax = fig.add_subplot(5,1,5)
            p.plot([start_date,end_date],[0,0],color=gray,linewidth=0.5)
            p.plot([start_date,end_date],[90,90],color=gray,linewidth=0.5)
            p.plot([start_date,end_date],[180,180],color=gray,linewidth=0.5)
            p.plot([start_date,end_date],[270,270],color=gray,linewidth=0.5)
            p.plot([start_date,end_date],[360,360],color=gray,linewidth=0.5)
            p.plot(df_stationdata_meteo.index,df_stationdata_meteo['WD'],'.',color='lightgray',label='wind dir')
            ax.set_xlim(start_date,end_date)
            ax.set_ylim(0,360)
            ax.set_ylabel('wind direction  [$^\circ$]')
            ax.grid(axis='x')
            #ax.grid(axis='y')
            ax.legend(loc='best',ncol=8)
    
            p.tight_layout()
            p.show()
            pngfile='selection_ICOS_'+ist+'_'+str(start_date.year)+str(start_date.month).zfill(2)
            fig.savefig(path_plots+pngfile+'.png',dpi=100)
            p.close()

        if summary:
            # print summary 
            summary_file='selection_ICOS_counts.csv'
            n_highco = df_obs_highco.groupby(pd.TimeGrouper(freq='M')).count()
            display(HTML('<br> <p style="font-size:15px;font-weight:bold;color:royalblue;">Events with CO offset > '+str(highco)+' ppm</p>'))
            display(HTML(n_highco['co'].to_frame(name='counts per month').to_html())) 
            open(summary_file,'w').write('Events with CO offset > '+str(highco)+' ppm')
            n_highco['co'].to_frame(name='counts per month').to_csv(summary_file,mode='a')
            
            ny_highco = n_highco['co2'].to_frame().groupby(n_highco['co2'].to_frame().index.month).mean()
            display(HTML('<br> <b>multi-annual mean</b>'))
            display(HTML(ny_highco['co2'].to_frame(name='counts per month').to_html())) 

            n_lowStdev = df_obs_lowStdev_noon.groupby(pd.TimeGrouper(freq='M')).count()
            display(HTML('<br> <p style="font-size:15px;font-weight:bold;color:royalblue;">Events with low CO2 variability < '+str(stdev_limit)+'ppm</p>'))
            display(HTML(n_lowStdev['Stdev'].to_frame(name='counts per month').to_html())) 
            open(summary_file,'a').write('Events with low CO2 variability < '+str(stdev_limit)+'ppm')
            n_lowStdev['co'].to_frame(name='counts per month').to_csv(summary_file,mode='a')

            ny_lowStdev = n_lowStdev['co2'].to_frame().groupby(n_lowStdev['co2'].to_frame().index.month).mean()
            display(HTML('<br> <b>multi-annual mean</b>'))
            display(HTML(ny_lowStdev['co2'].to_frame(name='counts per month').to_html())) 

            n_var = df_obs_var.groupby(pd.TimeGrouper(freq='M')).count()
            display(HTML('<br> <p style="font-size:15px;font-weight:bold;color:royalblue;">Events with small daytime CO2 difference < '+str(var_limit)+'ppm</p>'))
            display(HTML(n_var['co2'].to_frame(name='counts per month').to_html())) 
            open(summary_file,'a').write('Events with small daytime CO2 difference < '+str(var_limit)+'ppm')
            n_var['co'].to_frame(name='counts per month').to_csv(summary_file,mode='a')

            ny_var = n_var['co2'].to_frame().groupby(n_var['co2'].to_frame().index.month).mean()
            display(HTML('<br> <b>multi-annual mean</b>'))
            display(HTML(ny_var['co2'].to_frame(name='counts per month').to_html())) 


#### read coordiantes of big cities in Europe

In [14]:
filename='majorCitiesEurope.xlsx'
df_bigCities = pd.read_excel(filename,sheetname='Tabelle2',skiprows=1,
                             names=['Rang','Name','Einwohner','longitude','latitude','inhabitants'])
#problems with decimlal points... 
df_bigCities.inhabitants[df_bigCities.inhabitants > 100] = df_bigCities.inhabitants/1000.


In [15]:
def download_icos_data(station):
    ICOS_stations = [station[0:3]]
    
    ICOS_files_CO2 = get_ICOS_filename(ICOS_stations,tracer='CO2',download=True)
    ICOS_files_CO = get_ICOS_filename(ICOS_stations,tracer='CO',download=True)
    ICOS_files_MTO = get_ICOS_filename(ICOS_stations,tracer='MTO',download=True)

    ICOS_files = {'CO2': ICOS_files_CO2, 'CO': ICOS_files_CO, 'MTO': ICOS_files_MTO} 

    # adjust station name in case of differences between ICOS stationand STILT runs
    if station == 'GAT344':
        ICOS_station = 'GAT341'
    else:
        ICOS_station = station

    return ICOS_station, ICOS_files

In [16]:
def run_all(station_selection, ystart, mstart, yend, mend, 
            low_stilt, high_stilt, highco_stilt, var_limit_stilt,
            highco_obs, var_limit_obs, stdev_limit_obs):

    global_start_date = dt.datetime(ystart,mstart,1,0)
    global_end_date = dt.datetime(yend,mend,1,0)
    if (global_start_date > global_end_date):
        global_start_date = global_end_date
    if mend >= 12:
        global_end_date = dt.datetime(yend+1,1,1,0) 
    else:
        global_end_date = dt.datetime(yend,mend+1,1,0) 
    global_end_date = global_end_date - dt.timedelta(hours=1)
    if (global_start_date > global_end_date):
        print('Select Start Date < End Date')
        return
        
    for station in station_selection:

        loc_ident=stilt_stations[station]['locIdent']
        station_lat=stilt_stations[station]['lat']
        station_lon=stilt_stations[station]['lon']

        # selection of STILT time series (temporal and concentration thresholds)
        # only for those station for which 1-hourly results are availabe (see dropdown list)
        display(HTML('<p style="font-size:20px;font-weight:bold;color:royalblue;"> <br> Test sampling strategy on 1-hourly STILT time series </p>'))
        all_selection_plots(station,global_start_date,global_end_date,loc_ident,station_lat,station_lon,
                            high_stilt, low_stilt, highco_stilt, var_limit_stilt, summary=True)

        # download ICOS data 
        display(HTML('<p style="font-size:15px;font-weight:bold;color:royalblue;"> <br> Download ICOS observation time series from Carbon Portal </p>'))
        icos_station, icos_files = download_icos_data(station)
        
        display(HTML('<p style="font-size:20px;font-weight:bold;color:royalblue;"> <br> Comparison ICOS observation time series with STILT results </p>'))
        plot_comparison(icos_station,global_start_date,global_end_date,icos_files)
        
        # specify start and end date in case you want to analyse the same time period 
        display(HTML('<p style="font-size:20px;font-weight:bold;color:royalblue;"> <br> Apply selection to ICOS observation time series </p>'))
        plot_icos_ts_selection(icos_station,icos_files, 
                               highco_obs, var_limit_obs, stdev_limit_obs, 
                               start=global_start_date, end=global_end_date, summary=True)
        
        # if you want plots for the full time series at the ICOS site, use this call
        #plot_icos_ts_selection(icos_station,icos_files,
        #                       highco_obs, var_limit_obs, stdev_limit_obs)
    

In [17]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [18]:
def create_widget_stations():

    # find stations for which 1-hourly STILT results are available 
    allStations = sorted([stilt_stations[kk]['name'] for kk in os.listdir(path_stilt+'/Results_RINGO_T1.3/')])
    
    #Create a Dropdown widget with station names:
    station = Dropdown(options = allStations,
                       description = 'Station',
                       disabled= False,)
    
    #Create a Dropdown widget with year values (start year):
    s_year = Dropdown(options = [2017, 2018],
                      description = 'Start Year',
                      disabled= False,)
    
    #Create a Dropdown widget with month values (start month):
    s_month = Dropdown(options = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                       description = 'Start Month',
                       disabled= False,)
    
    #Create a Dropdown widget with year values (end year):
    e_year = Dropdown(options = [2017, 2018],
                      description = 'End Year',
                      disabled= False,)
    
    #Create a Dropdown widget with month values (end month):
    e_month = Dropdown(options = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                       description = 'End Month',
                       disabled= False,)
    
    #Create a Button widget to control execution:
    update_button = Button(description='Update',
                           disabled=False,
                           button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                           tooltip='Click me',)

    #Create a FloatText widget for STILT low fossil fuel CO2 threshold:
    low_stilt = FloatText(value=1.0,
                          description='Low ffCO2',
                          step=0.01,
                          description_long='Click me',
                          disabled=False)

    #Create a FloatText widget for STILT high fossil fuel CO2 threshold:
    high_stilt = FloatText(value=4.0,
                          description='High ffCO2',
                          step=0.01,
                          disabled=False)
    
    #Create a FloatText widget for STILT high CO threshold:
    highco_stilt = FloatText(value=0.04,
                          description='High ffCO',
                          step=0.01,
                          disabled=False)

    #Create a FloatText widget for STILT variability threshold:
    var_stilt = FloatText(value=1.0,
                          description='Var',
                          step=0.01,
                          disabled=False)
    
    #Create a FloatText widget for threshold of high CO in ICOS observation:
    highco_obs = FloatText(value=0.04,
                          description='High CO',
                          step=0.01,
                          disabled=False)

    #Create a FloatText widget for threshold of variability in ICOS observation:
    var_obs = FloatText(value=0.5,
                          description='Var',
                          step=0.01,
                          disabled=False)
    
    #Create a FloatText widget with threshold of variability in ICOS observation:
    stdev_obs = FloatText(value=1.0,
                          description='Stdev',
                          step=0.01,
                          disabled=False)
    
    header_station = Output()
    with header_station:
        display(HTML('<p style="font-size:15px;font-weight:bold;color:royalblue;">Select here station and time range: </p>'))

    header_stilt_1 = Output()
    with header_stilt_1:
        display(HTML('<p style="font-size:15px;font-weight:bold;color:royalblue;">Select here thresholds for STILT time series: </p>'))

    header_stilt_2 = Output()
    with header_stilt_2:
        display(HTML('Select here thresholds for high fossil fuel CO2 [ppm] and high fossil fuel CO [ppb] in STILT time series: '))

    header_stilt_3 = Output()
    with header_stilt_3:
        display(HTML('Select here thresholds for low fossil fuel CO2 [ppm] and low CO2 variability [ppm] in STILT time series: '))
    
    header_obs_1 = Output()
    with header_obs_1:
        display(HTML('<p style="font-size:15px;font-weight:bold;color:royalblue;">Select here thresholds for ICOS observation time series: </p>'))

    header_obs_2 = Output()
    with header_obs_2:
        display(HTML('Select here thresholds for high CO [ppb] in ICOS observation time series: '))

    header_obs_3 = Output()
    with header_obs_3:
        display(HTML('Select here thresholds for low CO2 variability during mid-day (Var) '+
                     'and standard deviation in noon-time measurement (Stdev) <br> '+
                     'in ICOS observation time series, both values in [ppm]: '))
    
    station_box = VBox([header_station,station])
    
    #Create a VBox for year and month:
    year_box = VBox([s_year, e_year])
    month_box = VBox([s_month, e_month])

    #Create a HBox for STILT thresholds:
    var_stilt_box = HBox([low_stilt, var_stilt])
    high_stilt_box = HBox([high_stilt, highco_stilt])

    #Add both time-related VBoxes to a HBox:
    time_box = HBox([year_box, month_box])

    #Add both STILT-related VBoxes to a HBox:
    #stilt_box = HBox([high_stilt_box, var_stilt_box])

    high_stilt_box_header = VBox([header_stilt_2, high_stilt_box])
    var_stilt_box_header = VBox([header_stilt_3, var_stilt_box])
    stilt_box = VBox([header_stilt_1, high_stilt_box_header, var_stilt_box_header])
    
    #Create a HBox for ICOS observation thresholds:
    var_obs_box = HBox([var_obs, stdev_obs])
    high_obs_box = HBox([highco_obs])
    
    high_obs_box_header = VBox([header_obs_2, high_obs_box])
    var_obs_box_header = VBox([header_obs_3, var_obs_box])
    obs_box = VBox([header_obs_1, high_obs_box_header, var_obs_box_header])    
    
    #Add all widgets to a VBox:
    form = VBox([station_box, time_box, stilt_box, obs_box, update_button])

    #Set font of all widgets in the form:
    station.layout.width = '623px'
    #update_button.style.button_color='#3973ac'
    update_button.style.button_color=royal
    time_box.layout.margin = '25px 0px 0px 0px'
    year_box.layout.margin = '0px 20px 0px 0px'
    stilt_box.layout.margin = '25px 0px 0px 0px'
    high_stilt_box_header.layout.margin = '5px 0px 0px 0px'
    var_stilt_box_header.layout.margin = '5px 0px 0px 0px'
    obs_box.layout.margin = '25px 0px 0px 0px'
    update_button.layout.margin = '50px 100px 40px 275px' #top, right, bottom, left

    #Initialize form output:
    form_out = Output()
    
    #Initialize results output:
    results_out = Output()

    
    #Define update function:
    def update_func(button_c):
        
        #Display selection:
        with results_out:
            
            #Clear previous results:
            clear_output()
            
            #Print "results" (current selection):
            print('Station: ', station.value)
            print('Start Year: ', s_year.value, '\t Start Month: ', s_month.value)
            print('End Year: ', e_year.value, '\t End Month: ', e_month.value)
            
            station_selection = [key for (key, value) in stilt_stations.items() if value['name'] == station.value]
            print('Station Selection: ',station_selection)

            print('threshold for low ffCO2 in STILT: ', low_stilt.value)
            print('threshold for high ffCO2 in STILT: ', high_stilt.value)
            print('threshold for high ffCO in STILT: ', highco_stilt.value)
            print('threshold for low variability in STILT: ', var_stilt.value)
            print('threshold for high CO in ICOS observations: ', highco_obs.value)
            print('threshold for low mid-day variability in ICOS CO2 observations: ', var_obs.value)
            print('threshold for low noon-time standard deviation in ICOS CO2 observations: ', stdev_obs.value)
            run_all(station_selection, s_year.value, s_month.value, e_year.value, e_month.value, 
                    low_stilt.value, high_stilt.value, highco_stilt.value, var_stilt.value,
                    highco_obs.value, var_obs.value, stdev_obs.value)

        
    #Call update-function when button is clicked:
    update_button.on_click(update_func)

    
    #Open form object:
    with form_out:
        
        #Clear previous selections in form:
        clear_output()
        
        #Display form and results:
        display(form, results_out)
        
    #Display form:
    display(form_out)

<a id='Top'></a>
### 1. Test sampling strategy on 1-hourly STILT time series
- Highest level at ICOS tower sites
- 1-hourly STILT results currently available for 2017 and 2018 at:
    - GAT 341m 
    - OPE 120m 
    - KRE 250m 
    - HPB 131m 
    - HTM 150m 
- selection criteria for QC
    - minimum variability of total CO2 during afternoon hours (12-15 local time)
    - use range between min and max in this time period
    - set threshold 'var_limit'
- selection of 'target' events
    - select high fossil fuel CO2 events (only available in model results)
        - set theshold 'high'
    - select background conditions with low fossil fuel CO2 (only available in model results)
        - set threshold 'low'
    - select CO offset (difference signal - background) as indicator for high fossil fuel CO2 event
        - background defined as lowest value in last 7 days
        - set theshold for offset 'highco'
- restrict selection to 13:00 local time (12 UTC) 
- aggregated footprints and contributions (emission x footprint)
    - monthly aggregation of 13:00 LT
    - low ffCO2
    - high ffCO2
    - high CO
    
### 2. Comparison ICOS observation time series with STILT results
- currently only ICOS level 2 data is included, downloaded from ICOS Carbon Portal to local folder
    - separate files for CO2, CO (and meteo)
    - GAT 341 m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
    - OPE 120 m for 2017 (Jan-Nov) and 2018 (Jan-Dec)
    - KRE 250m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
    - HPB 131m for 2017 (Feb-Dec) and 2018 (Jan-Dec)
    - HTM 150m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
- STILT results from standard footprint tool (3-hourly version) complemented with specific runs for RINGO Task1.3 (1-hourly)
- comparison of time series as monthly plots


### 3. Apply selection to ICOS observation time series
- Highest level at ICOS tower sites
    - GAT 341 m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
    - OPE 120 m for 2017 (Jan-Nov) and 2018 (Jan-Dec)
    - KRE 250m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
    - HPB 131m for 2017 (Feb-Dec) and 2018 (Jan-Dec)
    - HTM 150m for 2017 (Apr-Dec) and 2018 (Jan-Dec)
- selection criteria for QC (same as for model results)
    - minimum variability of total CO2 during afternoon hours (12-15 local time)
    - use range between min and max in this time period
    - set threshold 'var_limit'
- selection criteria for QC based on hourly standard deviation provided in ICOS data set
    - set threshold 'stdev_limit'
- selection of 'target' events
    - select CO offset (difference signal - background) as indicator for high fossil fuel CO2 event
        - background defined as lowest value in last 7 days
        - set theshold for offset 'highco'
- restrict selection to 13:00 local time (12 UTC)

<a id='Selection'></a>

[Back to top](#Top)

In [19]:
#Call function to present widget:
create_widget_stations()

Output()

[Back to selection](#Selection)


[Back to top](#Top)
